# 🚕 Taxi-v3 Reinforcement Learning — Q-Learning

This notebook trains a Q-Learning agent on the classic `Taxi-v3` environment, then exports the Q-table so you can load it into the interactive web UI.

**Steps:**
1. Install deps
2. Train the agent
3. Evaluate & visualize
4. Export `q_table.json` for the game UI

In [ ]:
!pip install gymnasium numpy matplotlib -q

In [ ]:
import gymnasium as gym
import numpy as np
import json
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

print('✅ Libraries loaded')

## 1 · Environment Overview

In [ ]:
env = gym.make('Taxi-v3', render_mode='ansi')
state, _ = env.reset()

print(f'Observation space : {env.observation_space}  ({env.observation_space.n} states)')
print(f'Action space      : {env.action_space}  ({env.action_space.n} actions)')
print()
print('Actions: 0=South  1=North  2=East  3=West  4=Pickup  5=Dropoff')
print()
print('Initial state render:')
print(env.render())

env.close()

## 2 · Hyperparameters

In [ ]:
# ── Feel free to tweak these ──────────────────────────────────────────────────
EPISODES       = 10_000   # Training episodes
ALPHA          = 0.10     # Learning rate
GAMMA          = 0.99     # Discount factor
EPSILON_START  = 1.0      # Initial exploration rate
EPSILON_END    = 0.01     # Minimum exploration rate
EPSILON_DECAY  = 0.0005   # Linear decay per episode
# ─────────────────────────────────────────────────────────────────────────────

print(f'Training for {EPISODES:,} episodes')
print(f'α={ALPHA}  γ={GAMMA}  ε: {EPSILON_START}→{EPSILON_END} (decay={EPSILON_DECAY})')

## 3 · Train

In [ ]:
env = gym.make('Taxi-v3')

n_states  = env.observation_space.n
n_actions = env.action_space.n
Q = np.zeros((n_states, n_actions))

epsilon = EPSILON_START
rewards_history = []
t_start = time.time()

for episode in range(1, EPISODES + 1):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = int(np.argmax(Q[state]))

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Bellman update
        Q[state, action] += ALPHA * (
            reward + GAMMA * np.max(Q[next_state]) - Q[state, action]
        )
        state = next_state
        total_reward += reward

    epsilon = max(EPSILON_END, epsilon - EPSILON_DECAY)
    rewards_history.append(total_reward)

    if episode % 1000 == 0:
        avg = np.mean(rewards_history[-1000:])
        elapsed = time.time() - t_start
        print(f'Ep {episode:>6} | Avg reward (1k): {avg:+7.2f} | ε={epsilon:.4f} | {elapsed:.0f}s')

env.close()
print(f'\n✅ Training done in {time.time()-t_start:.1f}s')

## 4 · Learning Curve

In [ ]:
def moving_avg(data, window=100):
    return np.convolve(data, np.ones(window)/window, mode='valid')

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
fig.patch.set_facecolor('#0a0a0f')

for ax in axes:
    ax.set_facecolor('#12121a')
    ax.tick_params(colors='#6b6b8a')
    ax.spines[:].set_color('#1e1e2e')

# Raw rewards
axes[0].plot(rewards_history, alpha=.2, color='#4dabf7', linewidth=.5)
axes[0].plot(moving_avg(rewards_history, 200), color='#f5c518', linewidth=1.5, label='MA-200')
axes[0].set_title('Reward per Episode', color='#e0e0f0')
axes[0].set_xlabel('Episode', color='#6b6b8a')
axes[0].set_ylabel('Total Reward', color='#6b6b8a')
axes[0].legend(facecolor='#12121a', labelcolor='#e0e0f0')

# Avg reward per 1000 episodes
chunk = 1000
chunked = [np.mean(rewards_history[i:i+chunk]) for i in range(0, len(rewards_history)-chunk+1, chunk)]
axes[1].bar(range(len(chunked)), chunked, color='#39d353', alpha=.7)
axes[1].set_title('Avg Reward per 1k Episodes', color='#e0e0f0')
axes[1].set_xlabel('1k Episode Block', color='#6b6b8a')
axes[1].set_ylabel('Avg Reward', color='#6b6b8a')

plt.tight_layout()
plt.savefig('training_curve.png', dpi=120, bbox_inches='tight', facecolor='#0a0a0f')
plt.show()
print('Chart saved as training_curve.png')

## 5 · Evaluate

In [ ]:
eval_env = gym.make('Taxi-v3')
eval_rewards, eval_steps = [], []

N_EVAL = 500
for _ in range(N_EVAL):
    s, _ = eval_env.reset()
    total, n_steps, done = 0, 0, False
    while not done:
        action = int(np.argmax(Q[s]))
        s, r, term, trunc, _ = eval_env.step(action)
        done = term or trunc
        total += r; n_steps += 1
    eval_rewards.append(total)
    eval_steps.append(n_steps)

eval_env.close()

print(f'=== Evaluation over {N_EVAL} episodes ===')
print(f'Avg reward : {np.mean(eval_rewards):+.2f}')
print(f'Avg steps  : {np.mean(eval_steps):.1f}')
print(f'Success (reward>0): {100*np.mean(np.array(eval_rewards)>0):.1f}%')

## 6 · Watch Agent Play (text render)

In [ ]:
render_env = gym.make('Taxi-v3', render_mode='ansi')
state, _ = render_env.reset()
done = False
ACTION_NAMES = ['⬇ South','⬆ North','➡ East','⬅ West','🧍 Pickup','📍 Dropoff']

for step in range(50):
    clear_output(wait=True)
    print(render_env.render())
    if done: print('✅ Done!'); break
    action = int(np.argmax(Q[state]))
    state, reward, terminated, truncated, _ = render_env.step(action)
    done = terminated or truncated
    print(f'Step {step+1}: {ACTION_NAMES[action]}  →  reward={reward:+d}')
    time.sleep(0.4)

render_env.close()

## 7 · Export Q-table → Load into Web UI

In [ ]:
with open('q_table.json', 'w') as f:
    json.dump(Q.tolist(), f)

print('✅ q_table.json saved!')
print(f'   Shape: {Q.shape}  ({Q.shape[0]} states × {Q.shape[1]} actions)')
print()
print('Next step:')
print('  1. Open  taxi_rl_game.html  in your browser')
print('  2. Click the file picker in the Agent panel')
print('  3. Select  q_table.json  — your trained agent will take over!')